In [ ]:
import os
import pandas as pd
import psycopg2

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

from dotenv import load_dotenv

In [ ]:
load_dotenv()
KEY=os.getenv("OPENAI_API_KEY")

In [ ]:
GENERATE_QUERY_TEMPLATE="""
Database Structure:{table_structure}

You are an expert postgres query builder. Given the above postgres database structure, it is your job to create a executable query to fetch the data for the following question. 

Question:{question}
"""

In [ ]:
USERS_TABLE_STRUCTURE="CREATE TABLE 'users' ('id' INTEGER NOT NULL DEFAULT 'nextval(''users_id_seq''::regclass)','created_at' TIMESTAMP NULL DEFAULT NULL,'updated_at' TIMESTAMP NULL DEFAULT NULL,'updated_by' INTEGER NOT NULL DEFAULT '0','status' CHAR(1) NOT NULL DEFAULT 'Y','code' CHAR(15) NULL DEFAULT NULL,'id_department' INTEGER NOT NULL DEFAULT '0','first_name' CHAR(255) NULL DEFAULT NULL,'middle_name' CHAR(255) NULL DEFAULT NULL,'last_name' CHAR(255) NULL DEFAULT NULL,'email_id' CHAR(125) NULL DEFAULT NULL,'gender' INTEGER NOT NULL DEFAULT '1','nationality' INTEGER NULL DEFAULT NULL,'date_of_birth' DATE NULL DEFAULT NULL,'marital_status' INTEGER NOT NULL DEFAULT '1','mobile_no' CHAR(20) NULL DEFAULT NULL,'department_id' CHAR(255) NULL DEFAULT NULL,PRIMARY KEY ('id'),);"

In [ ]:
def generate_query(prompt):
    
    llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.1)
    
    postgres_query_generator = PromptTemplate(
        input_variables=["table_structure","question"],
        template=GENERATE_QUERY_TEMPLATE
    )
    
    query_chain=LLMChain(llm=llm, prompt=postgres_query_generator, output_key="query", verbose=False)
    
    generate_query_chain=SequentialChain(
        chains=[query_chain], 
        input_variables=["table_structure", "question"],
        output_variables=["query"], 
        verbose=True
    )
    
    response_one=generate_query_chain(
        {
            "table_structure": USERS_TABLE_STRUCTURE,
            "question": prompt
        }
    )
    
    postgres_query = response_one['query']
    
    return postgres_query

In [ ]:
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USERNAME = os.getenv("DB_USERNAME")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [ ]:

def create_connection():
    connection = psycopg2.connect(
        host = DB_HOST,
        database = DB_NAME,
        user = DB_USERNAME,
        password = DB_PASSWORD
    )
    return connection

In [ ]:
def execute_query(connection, query):
    cursor = connection.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    cursor.close()
    return results

In [ ]:
def get_query_response(postgres_query):
    postgres_connection = create_connection()
    results = execute_query(postgres_connection, postgres_query)
    return results

In [ ]:
RESPONSE_TEMPLATE = """
You are an helping assistant. generate a response for question "{question}" using the {db_response}.
"""

In [ ]:
def generate_answer(prompt, results):
    
    llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)
    
    response_generator = PromptTemplate(
        input_variables=["question","db_response"],
        template=RESPONSE_TEMPLATE
    )
    
    response_chain=LLMChain(llm=llm, prompt=response_generator, output_key="answer", verbose=False)
    
    generate_response_chain=SequentialChain(
        chains=[response_chain], 
        input_variables=["question", "db_response"],
        output_variables=["answer"], 
        verbose=True
    )
    
    response_two=generate_response_chain(
        {
            "question": prompt,
            "db_response": results
        }
    )
    
    answer = response_two["answer"]
    
    return answer

In [ ]:
def chat_with_users(prompt):

    generated_query = generate_query(prompt)

    generated_query = generated_query.replace("\n", " ")

    generated_query

    query_db_response = get_query_response(generated_query)

    query_db_response

    answer = generate_answer(prompt, query_db_response)

    return answer

In [44]:
while True:
    input_prompt = input("Ask me about users table: ")
    if input_prompt == 'stop':
        break
    else:
        output = chat_with_users(input_prompt)
        print(input_prompt)
        print(output)
